In [23]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import csv
import re
from collections import defaultdict
from datetime import datetime

import sys
sys.path.append("./missing_data")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from load_data import get_files_from_folder
from data_stats import extract_dates_pattern_airmass_rgb_20200101_0000

In [18]:
# Percorso alla cartella dei frame
input_dir = "./download_data/from_gcloud"  # Cambia questo percorso
output_dir = "E:/Medicanes_Data/airmassRGB"  # Percorso per salvare i CSV
os.makedirs(output_dir, exist_ok=True)

In [4]:
filenames = get_files_from_folder(folder=input_dir, extension="png")

In [7]:
file_metadata = []
for fname in filenames:
    start_dt = extract_dates_pattern_airmass_rgb_20200101_0000(fname.name)
    file_metadata.append((fname, start_dt))

In [14]:
sorted_files = sorted(file_metadata, key=lambda x: x[1])  # Ordina per start_dt
#random_fnames =  [item[0] for item in file_metadata]
sorted_filenames = [item[0] for item in sorted_files]


In [16]:
#sorted_filenames

### copia le immagini in sottocartelle .../partN

In [24]:
def split_into_subfolders(images, output_dir, num_frames=16):
    subfolders = []
    num_total_files = len(images)

    # Numero di sottocartelle necessarie
    num_subfolders = num_total_files // num_frames

    for i in range(num_subfolders):
        subfolder_name = f"part{i+1}"
        subfolder_path = os.path.join(output_dir, subfolder_name)
        os.makedirs(subfolder_path, exist_ok=True)

        # Copia i file nella sottocartella con il nome 'img_{:05}.png'
        start_idx = i * num_frames
        end_idx = start_idx + num_frames

        for idx, file in enumerate(images[start_idx:end_idx]):
            new_name = os.path.join(subfolder_path, f"img_{idx+1:05d}.png")
            shutil.copy(file, new_name)

        subfolders.append(subfolder_path)

    return subfolders

In [25]:
subfolders = split_into_subfolders(sorted_filenames, output_dir)

### Suddivide in train/test/val e scrive i CSV

In [28]:
output_dir

'E:/Medicanes_Data/airmassRGB'

In [30]:
from build_dataset import create_csv

In [33]:
create_csv(output_dir)

File CSV generati:
Train: E:/Medicanes_Data/airmassRGB\train.csv
Test: E:/Medicanes_Data/airmassRGB\test.csv
Validation: E:/Medicanes_Data/airmassRGB\val.csv


In [29]:
total = len(subfolders)
train_split = int(total * 0.7)
test_split = int(total * 0.99)

# per riscrivere i file csv
subfolders = sorted([os.path.join(output_dir, d) for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))])


train_dirs = subfolders[:train_split]
test_dirs = subfolders[train_split:test_split]
val_dirs = subfolders[test_split:]

# Scrive nei file CSV con il formato richiesto
def write_to_csv(dirs, csv_file):
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["path", "start", "end"])  # Intestazione
        for dir_path in dirs:
            writer.writerow([dir_path, 1, 16])  # Riga nel formato richiesto

write_to_csv(train_dirs, train_csv)
write_to_csv(test_dirs, test_csv)
write_to_csv(val_dirs, val_csv)

print(f"File CSV generati:\nTrain: {train_csv}\nTest: {test_csv}\nValidation: {val_csv}")

File CSV generati:
Train: E:/Medicanes_Data/airmassRGB\train.csv
Test: E:/Medicanes_Data/airmassRGB\test.csv
Validation: E:/Medicanes_Data/airmassRGB\val.csv
